In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from datetime import datetime
import pypyodbc
import logging

# Configure logging
logging.basicConfig(filename="ccil_scraper.log", level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Connect to Azure SQL
def connect_to_db():
    try:
        print('attempting to connect to database..')
        conn = pypyodbc.connect(
            "Driver={ODBC Driver 18 for SQL Server};Server=tcp:modular-server.database.windows.net,1433;Database=modular;Uid=CloudSA1c5b822c;Pwd={Givemeinternship!};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
        ) #connection string that was obtained from azure server.
        print('returning database object...')
        return conn
    except pypyodbc.Error as e:
        logging.error(f"Database connection failed: {e}")
        return None

#scraping ccil market data
#selenium was used using a headless browser so that it can run on the virtual machine. beautiful soup cannot be used as the data from the table is loaded dynamically based on javascript.
def scrape_ccil():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    url = "https://www.ccilindia.com/web/ccil/rbi-nds-om1"
    driver.get(url)
    time.sleep(5)
    rows = driver.find_elements("xpath", "//table/tbody/tr") #extract all the relevant tables.
    data = []
    for row in rows:
        res = []
        cols = row.find_elements("tag name", "td")
        if len(cols) < 10:  
            continue
        for col in cols:
            res.append(col.text.strip())
        data.append(res)

    driver.quit()

    if not data:
        logging.warning("No data scraped from the website.")
        return None

    df = pd.DataFrame(data, columns=["Security", "Trades", "TTA", "Open", "High", "Low", "LTP", "Signal", "T/G", "LTY"])
    df["Timestamp"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    return df #return the dataframe extracted from the most recent page.

In [46]:

df = scrape_ccil()


In [47]:
df.head()

,Security,Trades,TTA,Open,High,Low,LTP,Signal,T/G,LTY,Timestamp
0,06.79 GS 2034,918,8745.00,100.6500,100.7650,100.6450,100.7000,,G,6.6888,2025-03-12 17:31:40
1,07.18 GS 2033,97,1285.00,102.6300,102.7150,102.6300,102.6900,,G,6.7546,2025-03-12 17:31:40
2,07.10 GS 2034,107,1215.00,102.2800,102.3400,102.2800,102.3150,,T,6.7534,2025-03-12 17:31:40
3,06.92 GS 2039,97,1060.00,100.5700,100.6700,100.5700,100.6675,,T,6.8457,2025-03-12 17:31:40
4,07.09 GS 2054,75,1035.00,100.3500,100.5300,100.3300,100.4700,,G,7.0511,2025-03-12 17:31:40


In [48]:
def connect_to_db():
    try:
        print('attempting to connect to database..')
        conn = pypyodbc.connect(
            "Driver={ODBC Driver 18 for SQL Server};Server=tcp:modular-server.database.windows.net,1433;Database=modular;Uid=CloudSA1c5b822c;Pwd={Givemeinternship!};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
        ) #connection string that was obtained from azure server.
        print('returning database object...')
        return conn
    except pypyodbc.Error as e:
        logging.error(f"Database connection failed: {e}")
        return None

In [49]:
conn = connect_to_db()
cursor = conn.cursor()


attempting to connect to database..
returning database object...


In [51]:
def row_exists(row, cursor):
    security = row["Security"]
    trades = row["Trades"]
    tta = row["TTA"]
    open = row["Open"]
    high = row["High"]
    low = row["Low"]
    ltp = row["LTP"]
    signal = row["Signal"]
    tg = row["T/G"]
    lty = row["LTY"]
    cursor.execute("""
    SELECT 1 FROM mktdata
    WHERE Security = ?
    AND Trades = ?
    AND TTA = ?
    AND [Open] = ?
    AND High = ?
    AND Low = ?
    AND LTP = ?
    AND SIGNAL = ?
    AND T_G = ?
    AND LTY = ?
    AND CONVERT(DATE, Timestamp) >= DATEADD(DAY, -1, CONVERT(DATE, GETDATE()))
    ORDER BY Timestamp DESC
    """, (security, trades, tta , open, high, low, ltp, signal, tg, lty))
    
    return cursor.fetchone() is not None #returns true if there are exact records found, otherwise returns false.
    

for index, row in df.iterrows():
    if row_exists(row, cursor):
        print("This row exists in the database")
        continue
    else:
        print(row)
        print("This above row doesn't exist in the database")
    


    

Security           06.79 GS 2034
Trades                       918
TTA                      8745.00
Open                    100.6500
High                    100.7650
Low                     100.6450
LTP                     100.7000
Signal                          
T/G                            G
LTY                       6.6888
Timestamp    2025-03-12 17:31:40
Name: 0, dtype: object
This above row doesn't exist in the database
Security           07.18 GS 2033
Trades                        97
TTA                      1285.00
Open                    102.6300
High                    102.7150
Low                     102.6300
LTP                     102.6900
Signal                          
T/G                            G
LTY                       6.7546
Timestamp    2025-03-12 17:31:40
Name: 1, dtype: object
This above row doesn't exist in the database
Security           07.10 GS 2034
Trades                       107
TTA                      1215.00
Open                    102.2800
High  